<a href="https://colab.research.google.com/github/quinnie-nguyen/Patent-Data-and-Social-Network-Analysis-using-R/blob/master/demo_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook uses cleaned data, i.e, a)drop all features that have more than 80% misssing values; b) drop rows that have more than 50% features missing; c)imputation by mean, mode, median.


Let's see how it works!


!!!!!!!!! minimum accuracy must be **78.3%**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from functools import partial
from collections import Counter
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
# load data
df = pd.read_csv('/content/gdrive/My Drive/WIDS/cleaned_data_0702.csv')
dictionary = pd.read_excel('/content/gdrive/My Drive/WIDS/DataDictionaryWiDS2021-fixed-thu-tu.xlsx')

In [ ]:
dictionary

,class,feature,value,type,description,example,rank
0,APACHE covariate,glucose_apache,mmol/L,numeric,The glucose concentration measured during the ...,5,1
1,labs,h1_glucose_max,mmol/L,numeric,The highest glucose concentration of the patie...,5,2
2,labs,h1_glucose_min,mmol/L,numeric,The lowest glucose concentration of the patien...,5,2
3,labs,d1_glucose_max,mmol/L,numeric,The highest glucose concentration of the patie...,5,3
4,labs,d1_glucose_min,mmol/L,numeric,The lowest glucose concentration of the patien...,5,3
5,demographic,gender,None,string,The genotypical sex of the patient,F,4
6,demographic,height,centimetres,numeric,The height of the person on unit admission,180,5
7,demographic,weight,kilograms,numeric,The weight (body mass) of the person on unit a...,80,5
8,demographic,ethnicity,None,string,The common national or cultural tradition whic...,Caucasian,6
9,vitals,d1_diasbp_invasive_max,Millimetres of mercury,numeric,The patient's highest diastolic blood pressure...,60,7


In [ ]:
con_fea = ['age','bmi','glucose_apache','h1_glucose_max','h1_glucose_min','d1_glucose_max','d1_glucose_min','d1_diasbp_invasive_max','d1_diasbp_invasive_min','d1_diasbp_max',
           'd1_diasbp_min','d1_diasbp_noninvasive_max','d1_diasbp_noninvasive_min','d1_sysbp_invasive_max','d1_sysbp_invasive_min','d1_sysbp_max','d1_sysbp_min',
           'd1_sysbp_noninvasive_max','d1_sysbp_noninvasive_min','heart_rate_apache','d1_heartrate_max','d1_heartrate_min','h1_diasbp_invasive_max','h1_diasbp_invasive_min',
           'h1_diasbp_max','h1_diasbp_min','h1_diasbp_noninvasive_max','h1_diasbp_noninvasive_min','h1_sysbp_invasive_max','h1_sysbp_invasive_min','h1_sysbp_max',
           'h1_sysbp_min','h1_sysbp_noninvasive_max','h1_sysbp_noninvasive_min','h1_heartrate_max','h1_heartrate_min','d1_albumin_max','d1_albumin_min',
           'h1_lactate_max','h1_lactate_min','d1_lactate_max','d1_lactate_min','d1_bilirubin_max','d1_bilirubin_min','h1_albumin_max','h1_albumin_min',
           'h1_bilirubin_max','h1_bilirubin_min','ethnicity_Asian','ethnicity_Caucasian','ethnicity_Hispanic','ethnicity_Native American','ethnicity_Other/Unknown',
           'gender_M']

remaining_fea = df.columns

selected_fea = [col for col in con_fea if col in remaining_fea]
selected_fea

['age',
 'bmi',
 'glucose_apache',
 'h1_glucose_max',
 'h1_glucose_min',
 'd1_glucose_max',
 'd1_glucose_min',
 'd1_diasbp_invasive_max',
 'd1_diasbp_invasive_min',
 'd1_diasbp_max',
 'd1_diasbp_min',
 'd1_diasbp_noninvasive_max',
 'd1_diasbp_noninvasive_min',
 'd1_sysbp_invasive_max',
 'd1_sysbp_invasive_min',
 'd1_sysbp_max',
 'd1_sysbp_min',
 'd1_sysbp_noninvasive_max',
 'd1_sysbp_noninvasive_min',
 'heart_rate_apache',
 'd1_heartrate_max',
 'd1_heartrate_min',
 'h1_diasbp_max',
 'h1_diasbp_min',
 'h1_diasbp_noninvasive_max',
 'h1_diasbp_noninvasive_min',
 'h1_sysbp_max',
 'h1_sysbp_min',
 'h1_sysbp_noninvasive_max',
 'h1_sysbp_noninvasive_min',
 'h1_heartrate_max',
 'h1_heartrate_min',
 'd1_albumin_max',
 'd1_albumin_min',
 'd1_lactate_max',
 'd1_lactate_min',
 'd1_bilirubin_max',
 'd1_bilirubin_min',
 'ethnicity_Asian',
 'ethnicity_Caucasian',
 'ethnicity_Hispanic',
 'ethnicity_Native American',
 'ethnicity_Other/Unknown',
 'gender_M']

In [ ]:
df.head()

,Unnamed: 0,age,bmi,elective_surgery,height,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2_apache,ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,ventilated_apache,wbc_apache,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,...,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown,gender_M,hospital_admit_source_Chest Pain Center,hospital_admit_source_Direct Admit,hospital_admit_source_Emergency Department,hospital_admit_source_Floor,hospital_admit_source_ICU,hospital_admit_source_ICU to SDU,hospital_admit_source_Observation,hospital_admit_source_Operating Room,hospital_admit_source_Other,hospital_admit_source_Other Hospital,hospital_admit_source_Other ICU,hospital_admit_source_PACU,hospital_admit_source_Recovery Room,hospital_admit_source_Step-Down Unit (SDU),icu_admit_source_Floor,icu_admit_source_Operating Room / Recovery,icu_admit_source_Other Hospital,icu_admit_source_Other ICU,icu_stay_type_readmit,icu_stay_type_transfer,icu_type_CSICU,icu_type_CTICU,icu_type_Cardiac ICU,icu_type_MICU,icu_type_Med-Surg ICU,icu_type_Neuro ICU,icu_type_SICU,diabetes_mellitus
0,0,68.0,22.732803,0,180.3,0.541667,0,73.9,2.300000,113.0,502.01,0,0,0.400000,31.000000,2.510000,0.608874,3.0,6.0,0.0,4.0,168.00000,118.0,27.400000,0,40.0,42.503378,42.503378,130.126947,7.352285,36.0,134.000000,39.3,1880.975174,0,14.100000,46.000000,32.000000,68.0,37.0,...,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1.0
1,1,77.0,27.421875,0,160.0,0.927778,0,70.2,2.842762,108.0,203.01,0,0,1.012062,9.000000,0.560000,1.000000,1.0,3.0,0.0,1.0,145.00000,120.0,36.900000,0,46.0,37.000000,37.000000,51.000000,7.450000,33.0,145.000000,35.1,1681.810832,1,12.700000,79.773308,46.013596,95.0,31.0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1.0
2,2,25.0,31.952749,0,172.7,0.000694,0,95.3,2.842762,122.0,703.03,0,0,1.012062,24.040124,1.240601,0.581356,3.0,6.0,0.0,5.0,159.31134,102.0,32.152284,0,68.0,42.335091,42.335091,131.026038,7.351192,37.0,138.004869,36.7,1681.810832,0,12.337758,79.773308,46.013596,88.0,48.0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0.0
3,3,81.0,22.635548,1,165.1,0.000694,0,61.7,2.842762,203.0,1206.03,1,0,1.012062,24.040124,1.240601,0.600000,4.0,6.0,0.0,5.0,185.00000,114.0,25.900000,1,60.0,30.000000,30.000000,142.000000,7.390000,4.0,138.004869,34.8,1681.810832,1,8.000000,62.000000,30.000000,48.0,42.0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0.0
4,4,67.0,27.555611,0,190.5,0.000694,0,100.0,2.932497,301.0,403.01,0,0,1.295918,13.000000,0.710000,0.608874,4.0,6.0,0.0,5.0,156.00000,113.0,44.200000,0,130.0,42.503378,42.503378,130.126947,7.352285,35.0,137.000000,36.6,1880.975174,0,10.900000,78.179184,46.954389,100.0,61.0,...,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1.0


In [ ]:
 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1, stratify=y)

# Building models applying Stratified Split!!!

In [ ]:
y = df.iloc[:,-1]

In [ ]:
X = df.loc[:, selected_fea]

In [ ]:
y.head()

0    1.0
1    1.0
2    0.0
3    0.0
4    1.0
Name: diabetes_mellitus, dtype: float64

In [ ]:
X.head()

,glucose_apache,h1_glucose_max,h1_glucose_min,d1_glucose_max,d1_glucose_min,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,heart_rate_apache,d1_heartrate_max,d1_heartrate_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,d1_albumin_max,d1_albumin_min,d1_lactate_max,d1_lactate_min,d1_bilirubin_max,d1_bilirubin_min,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown,gender_M
0,168.00000,164.830653,156.400769,168.000000,109.000000,46.000000,32.000000,68.0,37.0,68.0,37.0,122.000000,64.000000,131.0,73.0,131.0,73.0,118.0,119.0,72.0,68.0,63.0,68.000000,63.000000,131.0,115.0,131.000000,115.000000,119.0,108.0,2.300000,2.300000,1.300000,1.000000,0.400000,0.400000,0,1,0,0,0,1
1,145.00000,145.000000,143.000000,145.000000,128.000000,79.773308,46.013596,95.0,31.0,95.0,31.0,153.067895,92.752244,159.0,67.0,159.0,67.0,120.0,118.0,72.0,61.0,48.0,61.000000,48.000000,95.0,71.0,95.000000,71.000000,114.0,100.0,1.600000,1.600000,3.500000,3.500000,0.500000,0.500000,0,1,0,0,0,0
2,159.31134,167.721828,159.128092,172.667436,114.567355,79.773308,46.013596,88.0,48.0,88.0,48.0,153.067895,92.752244,148.0,105.0,148.0,105.0,102.0,96.0,68.0,88.0,58.0,88.000000,58.000000,148.0,124.0,148.000000,124.000000,96.0,78.0,2.907929,2.839910,2.836248,2.026176,1.013703,0.949435,0,1,0,0,0,0
3,185.00000,167.721828,159.128092,185.000000,88.000000,62.000000,30.000000,48.0,42.0,48.0,42.0,164.000000,78.000000,158.0,84.0,158.0,84.0,114.0,116.0,92.0,62.0,44.0,73.998018,60.032033,136.0,106.0,132.286834,114.660925,100.0,96.0,2.907929,2.839910,2.836248,2.026176,1.013703,0.949435,0,1,0,0,0,0
4,156.00000,164.830653,156.400769,156.000000,125.000000,78.179184,46.954389,100.0,61.0,100.0,61.0,152.420693,93.640280,173.0,107.0,173.0,107.0,113.0,113.0,83.0,89.0,89.0,89.000000,89.000000,143.0,143.0,143.000000,143.000000,83.0,83.0,2.998558,2.927553,2.871167,2.049455,1.283528,1.204542,0,1,0,0,0,1


In [ ]:
y.value_counts()

0.0    100699
1.0     27958
Name: diabetes_mellitus, dtype: int64

In [ ]:
# split imbalanced dataset into train and test sets with stratification
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
# create dataset
print(Counter(y))
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1, stratify=y)
print(Counter(y_train))
print(Counter(y_test))

Counter({0.0: 100699, 1.0: 27958})
Counter({0.0: 80559, 1.0: 22366})
Counter({0.0: 20140, 1.0: 5592})


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logisticRegr = LogisticRegression(solver = 'newton-cg', max_iter = 2000)
logisticRegr.fit(X_train, y_train)
y_pred = logisticRegr.predict(X_test)
accuracy_score(y_test, y_pred)

0.8049510337323177

In [ ]:
len(X.columns)

42

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.8133452510492771

In [ ]:
from xgboost import XGBClassifier
clf4 = XGBClassifier(max_depth=30, n_estimators=42,  objective='multi:softprob', learning_rate=0.3, num_class=2)
clf4.fit(X_train, y_train)
y_pred = clf4.predict(X_test)
accuracy_score(y_test, y_pred)

0.8070884501787657

In [ ]:
#from sklearn.neighbors import (NeighborhoodComponentsAnalysis, KNeighborsClassifier)
#ca = NeighborhoodComponentsAnalysis(random_state=42)
#nca.fit(X_train, y_train)
#y_nca = nca.predict(X_test)
#accuracy_score(y_test, y_nca)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_knn = knn.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_knn)

0.7895616353178921

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from functools import partial

models = list()
# create the pipeline
#pipe = Pipeline([('ContinuousFeturesSelector',ce), ('m', clf3)])
# add as a tuple to the list of models for voting
models.append(('rf', clf))
models.append(('knn', knn))
models.append(('lg', logisticRegr))

# define the voting ensemble

model = VotingClassifier(estimators=models, voting='soft', weights=[4,4,3])

model.fit(X_train.values, y_train)
print(f"accuracy score voting is {accuracy_score(y_test, model.predict(X_test.values))}")

accuracy score voting is 0.8096144877972952


In [ ]:
clf = RandomForestClassifier(
    n_estimators=50,
    criterion='gini',
    max_depth=5,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    bootstrap=True,
    oob_score=False,
    n_jobs=-1,
    random_state=0,
    verbose=0,
    warm_start=False,
    class_weight='balanced'
)

In [ ]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=5, random_state=123, shuffle=True)
results = pd.DataFrame(columns=['training_score', 'test_score'])
accuracy = []
    
for (train, test), i in zip(cv.split(X, y), range(5)):
    clf.fit(X.iloc[train], y.iloc[train])
    y_pred = clf.predict(X.iloc[test])
    accuracy.append(accuracy_score(y.iloc[test], y_pred))


In [ ]:
accuracy

[0.810624902844707,
 0.8101974195554174,
 0.8126384516730791,
 0.8098791341183786,
 0.813998678636664]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_knn))

              precision    recall  f1-score   support

         0.0       0.84      0.91      0.87     20140
         1.0       0.52      0.36      0.42      5592

    accuracy                           0.79     25732
   macro avg       0.68      0.63      0.65     25732
weighted avg       0.77      0.79      0.77     25732



# Building models based on resample dataset (balance the data wrt target)

## Oversampling method

In [ ]:
X = df.loc[:, selected_fea]
y = df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1, stratify=y)

(128657, 42)

In [ ]:
y.value_counts()/len(y)

0.0    0.782694
1.0    0.217306
Name: diabetes_mellitus, dtype: float64

In [ ]:
y_train.value_counts()/len(y_train)

0.0    0.782696
1.0    0.217304
Name: diabetes_mellitus, dtype: float64

In [ ]:
 pip install imbalanced-learn

In [ ]:
from imblearn.over_sampling import RandomOverSampler
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')
# fit and apply the transform
X_over, y_over = oversample.fit_resample(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
from collections import Counter
print(Counter(y_over))

Counter({0.0: 80559, 1.0: 80559})


In [ ]:
X_over = pd.DataFrame(X_over)
X_over.columns = X.columns
X_over.head()

,age,bmi,glucose_apache,h1_glucose_max,h1_glucose_min,d1_glucose_max,d1_glucose_min,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,heart_rate_apache,d1_heartrate_max,d1_heartrate_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,d1_albumin_max,d1_albumin_min,d1_lactate_max,d1_lactate_min,d1_bilirubin_max,d1_bilirubin_min,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown,gender_M
0,78.0,25.448202,394.0,164.830653,156.400769,394.0,184.0,78.179184,46.954389,72.0,68.0,72.0,68.0,152.420693,93.640280,130.0,118.0,130.0,118.0,176.0,88.0,73.0,68.0,68.0,68.000000,68.000000,118.0,118.0,118.000000,118.000000,88.0,88.0,3.000000,3.000000,1.320000,1.320000,0.400000,0.400000,0.0,1.0,0.0,0.0,0.0,1.0
1,44.0,33.214352,165.0,120.000000,120.000000,165.0,120.0,66.000000,52.000000,69.0,56.0,69.0,56.0,138.000000,97.000000,137.0,105.0,137.0,105.0,107.0,104.0,71.0,65.0,52.0,76.423325,63.834716,135.0,113.0,132.356291,115.518627,95.0,71.0,2.998558,2.927553,1.800000,0.800000,1.283528,1.204542,0.0,1.0,0.0,0.0,0.0,1.0
2,71.0,22.955862,169.0,231.000000,120.000000,231.0,120.0,83.419563,50.503905,98.0,56.0,98.0,56.0,158.856576,96.307315,141.0,98.0,141.0,98.0,106.0,110.0,82.0,98.0,63.0,98.000000,63.000000,141.0,110.0,141.000000,110.000000,110.0,84.0,2.100000,2.100000,3.500000,1.900000,0.600000,0.600000,0.0,0.0,0.0,0.0,0.0,1.0
3,46.0,15.937719,76.0,90.000000,90.000000,142.0,64.0,79.773308,46.013596,89.0,56.0,89.0,56.0,153.067895,92.752244,115.0,90.0,115.0,90.0,108.0,109.0,89.0,65.0,63.0,65.000000,63.000000,95.0,92.0,95.000000,92.000000,99.0,96.0,2.000000,2.000000,0.800000,0.800000,0.300000,0.300000,0.0,1.0,0.0,0.0,0.0,0.0
4,69.0,22.173655,99.0,167.721828,159.128092,145.0,99.0,79.773308,46.013596,87.0,40.0,87.0,40.0,153.067895,92.752244,129.0,72.0,129.0,72.0,90.0,91.0,72.0,64.0,57.0,64.000000,57.000000,118.0,90.0,118.000000,90.000000,81.0,75.0,2.907929,2.839910,2.836248,2.026176,1.013703,0.949435,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
df['diabetes_mellitus'].value_counts()

0.0    100699
1.0     27958
Name: diabetes_mellitus, dtype: int64

In [ ]:
#from sklearn.utils import resample

#tr_majority = df[df['diabetes_mellitus'] == 0]
#tr_minority = df[df['diabetes_mellitus'] == 1]

#tr_minority_upsampled = resample(tr_minority,
                                 replace=True,  # sample with replacement
                                 n_samples=100000,  # to match majority class 84000
                                 random_state=1)  # reproducible results

# Combine majority class with upsampled minority class
#tr_upsampled = pd.concat([tr_majority, tr_minority_upsampled])

# Display new class counts
#tr_upsampled.diabetes_mellitus.value_counts()


In [ ]:
#X_upsampled = tr_upsampled.loc[:,selected_fea]
#y_upsampled = tr_upsampled.iloc[:,-1]
#from sklearn.model_selection import train_test_split
# split into train test sets
#X_train, X_test, y_train, y_test = train_test_split(X_upsampled, y_upsampled, test_size=0.20, random_state=1)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logisticRegr = LogisticRegression(solver = 'newton-cg', max_iter = 2000)
logisticRegr.fit(X_over, y_over)
y_pred = logisticRegr.predict(X_test)
accuracy_score(y_test, y_pred)

0.7557515933468055

!!! Probably linear classificator is NOT suitable for our problem 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_over, y_over)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.8129566298771957

RF

*   oversampling the whole dataset --> 85.6%
*   oversampling just training set --> 81.05%


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_over, y_over)
y_knn = knn.predict(X_test)
accuracy_score(y_test, y_knn)

0.6828462614643246


*   78.5%
*   68.3%



In [ ]:
from xgboost import XGBClassifier
clf4 = XGBClassifier(max_depth=30, n_estimators=42,  objective='multi:softprob', learning_rate=0.3, num_class=2)
clf4.fit(X_over, y_over)
y_pred = clf4.predict(X_test)
accuracy_score(y_test, y_pred)

0.8039794808021141

XGBC

*   oversampling the whole dataset --> 92.6%
*   oversampling just training set --> 80.4%


In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from functools import partial
from sklearn.metrics import classification_report


models = list()
# create the pipeline
#pipe = Pipeline([('ContinuousFeturesSelector',ce), ('m', clf3)])
# add as a tuple to the list of models for voting
models.append(('rf', clf))
models.append(('knn', knn))
models.append(('XGBC', clf4))
models.append(('lg', logisticRegr))

# define the voting ensemble

model = VotingClassifier(estimators=models, voting='soft', weights=[3,2,3,2])

model.fit(X_over.values, y_over)
print(f"accuracy score voting is {accuracy_score(y_test, model.predict(X_test.values))}")
print(classification_report(y_test, model.predict(X_test.values)))

accuracy score voting is 0.8031633763407431
              precision    recall  f1-score   support

         0.0       0.87      0.88      0.88     20140
         1.0       0.55      0.52      0.53      5592

    accuracy                           0.80     25732
   macro avg       0.71      0.70      0.70     25732
weighted avg       0.80      0.80      0.80     25732



VC


*   Oversampling the whole : 92.22%


##Undersampling method

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy='majority')
# fit and apply the transform
X_under, y_under = undersample.fit_resample(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_under = pd.DataFrame(X_under)
X_under.columns = X_train.columns

In [ ]:
print(len(y_under))
print(len(y))
print(Counter(y_under))

55916
128657
Counter({0.0: 27958, 1.0: 27958})


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logisticRegr = LogisticRegression(solver = 'newton-cg', max_iter = 2000)
logisticRegr.fit(X_under, y_under)
y_log = logisticRegr.predict(X_test)
accuracy_score(y_test, y_log)

0.7545857298305612

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_under, y_under)
y_rf = clf.predict(X_test)
accuracy_score(y_test, y_rf)

0.8546556816415358

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_under, y_under)
y_knn = knn.predict(X_test)
accuracy_score(y_test, y_knn)

0.7465801336856832

In [ ]:
from xgboost import XGBClassifier
clf4 = XGBClassifier(max_depth=30, n_estimators=42,  objective='multi:softprob', learning_rate=0.3, num_class=2)
clf4.fit(X_under, y_under)
y_xgbc = clf4.predict(X_test)
accuracy_score(y_test, y_xgbc)

0.8447069796362506

84.62 without age and bmi

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from functools import partial
from sklearn.metrics import classification_report


models = list()
# create the pipeline
#pipe = Pipeline([('ContinuousFeturesSelector',ce), ('m', clf3)])
# add as a tuple to the list of models for voting
models.append(('rf', clf))
models.append(('knn', knn))
models.append(('XGBC', clf4))
models.append(('lg', logisticRegr))

# define the voting ensemble

model = VotingClassifier(estimators=models, voting='soft', weights=[4,1,4,1])

model.fit(X_under.values, y_under)
print(f"accuracy score voting is {accuracy_score(y_test, model.predict(X_test.values))}")
print(classification_report(y_test, model.predict(X_test.values)))

accuracy score voting is 0.8539561635317892
              precision    recall  f1-score   support

         0.0       1.00      0.81      0.90     20140
         1.0       0.60      1.00      0.75      5592

    accuracy                           0.85     25732
   macro avg       0.80      0.91      0.82     25732
weighted avg       0.91      0.85      0.86     25732



##Combining Random Oversampling and Undersampling

In [ ]:
y_train.value_counts()

0.0    80559
1.0    22366
Name: diabetes_mellitus, dtype: int64

In [ ]:
over = RandomOverSampler(sampling_strategy=0.4)
X_com, y_com = over.fit_resample(X_train, y_train)
print(Counter(y_com))
under = RandomUnderSampler(sampling_strategy=0.6)
X_com, y_com = under.fit_resample(X_com, y_com)
print(Counter(y_com))

Counter({0.0: 80559, 1.0: 32223})
Counter({0.0: 53705, 1.0: 32223})


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_com = pd.DataFrame(X_com)
X_com.columns = X_train.columns
X_com.head()

,age,bmi,glucose_apache,h1_glucose_max,h1_glucose_min,d1_glucose_max,d1_glucose_min,d1_diasbp_invasive_max,d1_diasbp_invasive_min,d1_diasbp_max,d1_diasbp_min,d1_diasbp_noninvasive_max,d1_diasbp_noninvasive_min,d1_sysbp_invasive_max,d1_sysbp_invasive_min,d1_sysbp_max,d1_sysbp_min,d1_sysbp_noninvasive_max,d1_sysbp_noninvasive_min,heart_rate_apache,d1_heartrate_max,d1_heartrate_min,h1_diasbp_max,h1_diasbp_min,h1_diasbp_noninvasive_max,h1_diasbp_noninvasive_min,h1_sysbp_max,h1_sysbp_min,h1_sysbp_noninvasive_max,h1_sysbp_noninvasive_min,h1_heartrate_max,h1_heartrate_min,d1_albumin_max,d1_albumin_min,d1_lactate_max,d1_lactate_min,d1_bilirubin_max,d1_bilirubin_min,ethnicity_Asian,ethnicity_Caucasian,ethnicity_Hispanic,ethnicity_Native American,ethnicity_Other/Unknown,gender_M
0,66.0,20.414982,77.00000,102.000000,102.000000,102.0,102.0,83.419563,50.503905,67.0,57.0,67.0,57.0,158.856576,96.307315,92.0,84.0,92.0,84.0,124.0,121.0,66.0,78.000000,52.000000,78.000000,52.000000,105.000000,70.000000,105.000000,70.000000,74.0,66.0,3.000000,3.00000,4.470000,2.350000,0.400000,0.400000,0.0,0.0,0.0,0.0,0.0,1.0
1,57.0,41.470491,151.00000,132.000000,132.000000,151.0,132.0,56.000000,48.000000,88.0,61.0,88.0,61.0,150.000000,113.000000,153.0,115.0,153.0,115.0,117.0,116.0,66.0,75.807118,63.248274,76.423325,63.834716,132.552247,115.253983,132.356291,115.518627,112.0,112.0,2.900000,2.90000,2.871167,2.049455,1.100000,1.100000,0.0,1.0,0.0,0.0,0.0,1.0
2,81.0,21.645014,162.73474,174.909719,165.116524,135.0,135.0,74.000000,48.000000,77.0,57.0,77.0,57.0,176.000000,112.000000,164.0,109.0,164.0,109.0,50.0,82.0,44.0,73.000000,66.000000,73.000000,66.000000,154.000000,136.000000,154.000000,136.000000,82.0,62.0,3.300000,3.30000,3.209862,2.199921,1.500000,1.500000,0.0,0.0,0.0,0.0,0.0,1.0
3,71.0,29.923671,67.00000,75.000000,75.000000,134.0,67.0,79.773308,46.013596,98.0,38.0,98.0,38.0,153.067895,92.752244,130.0,97.0,130.0,97.0,60.0,76.0,60.0,57.000000,38.000000,57.000000,38.000000,120.000000,97.000000,120.000000,97.000000,64.0,60.0,2.600000,2.60000,2.836248,2.026176,1.300000,1.300000,0.0,1.0,0.0,0.0,0.0,0.0
4,87.0,23.502142,75.00000,75.000000,75.000000,141.0,75.0,79.773308,46.013596,83.0,45.0,83.0,45.0,153.067895,92.752244,164.0,97.0,164.0,97.0,49.0,75.0,0.0,59.000000,54.000000,59.000000,54.000000,140.000000,139.000000,140.000000,139.000000,63.0,36.0,2.907929,2.83991,2.836248,2.026176,1.013703,0.949435,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
logisticRegr = LogisticRegression(solver = 'newton-cg', max_iter = 2000)
logisticRegr.fit(X_com, y_com)
y_log = logisticRegr.predict(X_test)
accuracy_score(y_test, y_log)

0.8007927871910462

LG:

*   80.32 (without age and bmi)
*   List item



In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_com, y_com)
y_rf = clf.predict(X_test)
accuracy_score(y_test, y_rf)

0.8099253847349603

RF


*   80.59
*   List item



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_com, y_com)
y_knn = knn.predict(X_test)
accuracy_score(y_test, y_knn)

0.7441706824187782

In [ ]:
from xgboost import XGBClassifier
clf4 = XGBClassifier(max_depth=30, n_estimators=42,  objective='multi:softprob', learning_rate=0.3, num_class=2)
clf4.fit(X_com, y_com)
y_xgbc = clf4.predict(X_test)
accuracy_score(y_test, y_xgbc)

0.8000932690812995

XGB 

*   79.737
*   List item



In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from functools import partial
from sklearn.metrics import classification_report


models = list()
# create the pipeline
#pipe = Pipeline([('ContinuousFeturesSelector',ce), ('m', clf3)])
# add as a tuple to the list of models for voting
models.append(('rf', clf))
models.append(('knn', knn))
models.append(('XGBC', clf4))
models.append(('lg', logisticRegr))

# define the voting ensemble

model = VotingClassifier(estimators=models, voting='soft', weights=[4,1,2,3])

model.fit(X_com.values, y_com)
print(f"accuracy score voting is {accuracy_score(y_test, model.predict(X_test.values))}")
print(classification_report(y_test, model.predict(X_test.values)))

accuracy score voting is 0.8096533499145033
              precision    recall  f1-score   support

         0.0       0.88      0.88      0.88     20140
         1.0       0.56      0.56      0.56      5592

    accuracy                           0.81     25732
   macro avg       0.72      0.72      0.72     25732
weighted avg       0.81      0.81      0.81     25732



VC


*   80.724
*   Precision (0.56 and 0.87)



In [ ]:
from sklearn.naive_bayes import GaussianNB
GNB = GaussianNB()
GNB.fit(X_com, y_com)
y_gnb = GNB.predict(X_test)
print(accuracy_score(y_test, y_gnb))
print(classification_report(y_test, y_gnb))

0.7868412871133219
              precision    recall  f1-score   support

         0.0       0.85      0.88      0.87     20140
         1.0       0.51      0.44      0.47      5592

    accuracy                           0.79     25732
   macro avg       0.68      0.66      0.67     25732
weighted avg       0.78      0.79      0.78     25732



In [ ]:
from sklearn.naive_bayes import GaussianNB
GNB = GaussianNB()
GNB.fit(X_over, y_over)
y_gnb = GNB.predict(X_test)
print(accuracy_score(y_test, y_gnb))
print(classification_report(y_test, y_gnb))

0.7796906575470232
              precision    recall  f1-score   support

         0.0       0.86      0.86      0.86     20140
         1.0       0.49      0.49      0.49      5592

    accuracy                           0.78     25732
   macro avg       0.68      0.67      0.68     25732
weighted avg       0.78      0.78      0.78     25732



In [ ]:
from sklearn.naive_bayes import GaussianNB
GNB = GaussianNB()
GNB.fit(X_under, y_under)
y_gnb = GNB.predict(X_test)
print(accuracy_score(y_test, y_gnb))
print(classification_report(y_test, y_gnb))

0.7731229597388466
              precision    recall  f1-score   support

         0.0       0.86      0.85      0.85     20140
         1.0       0.48      0.49      0.49      5592

    accuracy                           0.77     25732
   macro avg       0.67      0.67      0.67     25732
weighted avg       0.78      0.77      0.77     25732



In [ ]:
from sklearn import svm
svc = svm.SVC()
svc.fit(X_under, y_under)
y_svc = svc.predict(X_test)
print(accuracy_score(y_test, y_svc))
print(classification_report(y_test, y_svc))

0.7452976838178144
              precision    recall  f1-score   support

         0.0       0.91      0.75      0.82     20140
         1.0       0.45      0.74      0.56      5592

    accuracy                           0.75     25732
   macro avg       0.68      0.74      0.69     25732
weighted avg       0.81      0.75      0.76     25732



### SMOTE Oversampling method

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE


In [ ]:
from numpy import mean
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE

# define pipeline
steps = [('over', SMOTE()), ('model', DecisionTreeClassifier())]
pipeline = Pipeline(steps=steps)
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(pipeline, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('Mean ROC AUC: %.3f' % mean(scores))

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


Mean ROC AUC: 0.631


In [ ]:
from imblearn.over_sampling import BorderlineSMOTE
oversample = BorderlineSMOTE()
X_bsmote, y_bsmote = oversample.fit_resample(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_bsmote = pd.DataFrame(X_bsmote)
X_bsmote.columns = X_train.columns
X_bsmote.shape

(161118, 44)

In [ ]:
from sklearn.naive_bayes import GaussianNB
GNB = GaussianNB()
GNB.fit(X_bsmote, y_bsmote)
y_gnb = GNB.predict(X_test)
print(accuracy_score(y_test, y_gnb))
print(classification_report(y_test, y_gnb))

0.6968754857764651
              precision    recall  f1-score   support

         0.0       0.88      0.71      0.79     20140
         1.0       0.38      0.65      0.48      5592

    accuracy                           0.70     25732
   macro avg       0.63      0.68      0.63     25732
weighted avg       0.77      0.70      0.72     25732



In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_bsmote, y_bsmote)
y_knn = knn.predict(X_test)
print(accuracy_score(y_test, y_knn))
print(classification_report(y_test, y_knn))

0.685916368723768
              precision    recall  f1-score   support

         0.0       0.89      0.68      0.77     20140
         1.0       0.38      0.69      0.49      5592

    accuracy                           0.69     25732
   macro avg       0.63      0.69      0.63     25732
weighted avg       0.78      0.69      0.71     25732



In [ ]:
y_train_knn = knn.predict(X_bsmote)
accuracy_score(y_train_knn, y_bsmote)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_bsmote, y_bsmote)
y_rf = clf.predict(X_test)
accuracy_score(y_test, y_rf)

In [ ]:
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
over = SMOTE(sampling_strategy=0.4)
under = RandomUnderSampler(sampling_strategy=0.6)
steps = [('o', over), ('u', under)]
pipeline = Pipeline(steps=steps)
X_scom, y_scom = pipeline.fit_resample(X_train, y_train)
counter = Counter(y_scom)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/loca

In [ ]:
print(y_train.value_counts())
print(counter)
X_scom = pd.DataFrame(X_scom)
X_scom.columns = X_train.columns

0.0    80559
1.0    22366
Name: diabetes_mellitus, dtype: int64
Counter({0.0: 53705, 1.0: 32223})


In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
clf.fit(X_scom, y_scom)
y_rf = clf.predict(X_test)
print(accuracy_score(y_test, y_rf))
print(classification_report(y_test, y_rf))
print(classification_report(y_test, y_rf))

0.8074770713508472

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_scom, y_scom)
y_knn = knn.predict(X_test)
print(accuracy_score(y_test, y_knn))
print(classification_report(y_test, y_knn))

0.7347271879371988
              precision    recall  f1-score   support

         0.0       0.87      0.77      0.82     20140
         1.0       0.42      0.60      0.49      5592

    accuracy                           0.73     25732
   macro avg       0.65      0.69      0.66     25732
weighted avg       0.78      0.73      0.75     25732



In [ ]:
from xgboost import XGBClassifier
clf4 = XGBClassifier(max_depth=30, n_estimators=42,  objective='multi:softprob', learning_rate=0.3, num_class=2)
clf4.fit(X_scom, y_scom)
y_xgbc = clf4.predict(X_test)
print(accuracy_score(y_test, y_xgbc))
print(classification_report(y_test, y_xgbc))

0.7990828540338878
              precision    recall  f1-score   support

         0.0       0.88      0.86      0.87     20140
         1.0       0.54      0.56      0.55      5592

    accuracy                           0.80     25732
   macro avg       0.71      0.71      0.71     25732
weighted avg       0.80      0.80      0.80     25732



In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from functools import partial
from sklearn.metrics import classification_report


models = list()
# create the pipeline
#pipe = Pipeline([('ContinuousFeturesSelector',ce), ('m', clf3)])
# add as a tuple to the list of models for voting
models.append(('rf', clf))
models.append(('knn', knn))
models.append(('XGBC', clf4))


# define the voting ensemble

model = VotingClassifier(estimators=models, voting='soft', weights=[4,2,4])

model.fit(X_scom.values, y_scom)
print(f"accuracy score voting is {accuracy_score(y_test, model.predict(X_test.values))}")
print(classification_report(y_test, model.predict(X_test.values)))

accuracy score voting is 0.8040960671537385
              precision    recall  f1-score   support

         0.0       0.88      0.87      0.87     20140
         1.0       0.55      0.57      0.56      5592

    accuracy                           0.80     25732
   macro avg       0.71      0.72      0.72     25732
weighted avg       0.81      0.80      0.81     25732

